In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import pandas as pd
from xgboost import XGBRegressor, XGBRFRegressor 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.decomposition import PCA

%matplotlib notebook

In [2]:
X_TRAIN_PATH = './Case_material/train/X_final.csv'
Y_TRAIN_PATH = './Case_material/train/y_train.csv'

X = pd.read_csv(X_TRAIN_PATH)
y = pd.read_csv(Y_TRAIN_PATH)

X.drop(columns=['ValueDateTimeUTC', 'time'], inplace=True)
# X.drop(columns=['ValueDateTimeUTC'], inplace=True)
y.drop(columns=['ValueDateTimeUTC'], inplace=True)

# pca = PCA(n_components=25)
# pca.fit(X)
# X = pca.transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [224]:
class XGB_per_hour:
    def __init__(self):
        self.models = list()
        for i in range(24):
            xgb_model = XGBRegressor()
            self.models.append(xgb_model)
        
    def fit(self, X_train, y_train):
        for i in range(24):
            print("Fitting model ", i)
            hX_train = X_train.loc[X_train['hour'] == i]
            hy_train = y_train.loc[X_train['hour'] == i]
            self.models[i].fit(hX_train, hy_train)
    
    def predict(self, X_test):        
        all_preds = pd.DataFrame()
        orig_indices = X_test.index.values
        for i in range(24):
            df_slice = X_test[X_test['hour'] == i]
            indices = X_test[X_test['hour'] == i].index.values
            
            assert(df_slice.shape[0] == indices.shape[0])
            
            preds = pd.DataFrame(self.models[i].predict(df_slice), index=indices)
            all_preds.iloc[indices] = preds
            #             all_preds = all_preds.append(preds)
            
#         print(all_preds.iloc[[orig_indices]])
        return all_preds

    def predict_iter(self, X_test):
        preds = pd.DataFrame()
        
        for (index, row) in X_test.iterrows():
            print(index)
            print(row)
            temp_row = pd.DataFrame(row, index=[index])
            
            print(temp_row)
            
            preds = self.models[1].predict(temp_row)
#             print(temp_df)
            print(preds)
            break
            
        return preds
    
    
    def evaluate(self, X_test, y_test):        
        mses = []
        mapes = []
        mape_accs = []

        for i in range(24):
            print("Evaluating Model ", i)
            hX_test = X_test.loc[X_test['hour'] == i]
            hy_test = y_test.loc[X_test['hour'] == i]
            
            h_preds = self.models[i].predict(hX_test)

            mse = round(mean_squared_error(hy_test.to_numpy(), h_preds.reshape((-1, 1))),2)
            mape = np.mean(np.abs((hy_test.to_numpy() - h_preds.reshape((-1,1))) / np.abs(hy_test.to_numpy())))
            mape_acc = round(100*(1 - mape), 2)

            mses.append(mse)
            mapes.append(mape)
            mape_accs.append(mape_acc)
            
        return np.mean(mses), np.mean(mapes), np.mean(mape_accs)

In [225]:
xgb_ph = XGB_per_hour()

xgb_ph.fit(X_train, y_train)

Fitting model  0
Fitting model  1
Fitting model  2
Fitting model  3
Fitting model  4
Fitting model  5
Fitting model  6
Fitting model  7
Fitting model  8
Fitting model  9
Fitting model  10
Fitting model  11
Fitting model  12
Fitting model  13
Fitting model  14
Fitting model  15
Fitting model  16
Fitting model  17
Fitting model  18
Fitting model  19
Fitting model  20
Fitting model  21
Fitting model  22
Fitting model  23


In [226]:
preds = xgb_ph.predict_iter(X_test)

print(preds.shape)
print(X_test.shape)

# print(preds.iloc[[X_test.index.values]])
# print(preds.head())
# print(y_test.head())

# preds = preds.to_numpy()
# mse = round(mean_squared_error(y_test.to_numpy(), preds.reshape((-1, 1))),2)
# mape = np.mean(np.abs((y_test.to_numpy() - preds.reshape((-1,1))) / np.abs(y_test.to_numpy())))
# mape_acc = round(100*(1 - mape), 2)

# print("\n===================")
# print("MSE: ", mse)
# print("MAPE: ", mape)
# print("Acc.: ", mape_acc)

6070
Madrid_d2m          0.213180
Madrid_t2m          0.197163
Madrid_i10fg        0.043752
Madrid_sp           0.084996
Madrid_tcc          0.011750
Madrid_tp           0.000000
Barcelona_d2m       0.232712
Barcelona_t2m       0.148898
Barcelona_i10fg     0.029513
Barcelona_sp        0.218627
Barcelona_tcc       0.179113
Barcelona_tp        0.004163
Valencia_d2m        0.133529
Valencia_t2m        0.085433
Valencia_i10fg      0.031668
Valencia_sp         0.106032
Valencia_tcc        0.074814
Valencia_tp         0.000123
Seville_d2m         0.083932
Seville_t2m         0.063919
Seville_i10fg       0.016573
Seville_sp          0.088590
Seville_tcc         0.000000
Seville_tp          0.000000
Zaragoza_d2m        0.034777
Zaragoza_t2m        0.024390
Zaragoza_i10fg      0.007744
Zaragoza_sp         0.023451
Zaragoza_tcc        0.020150
Zaragoza_tp         0.000066
Malaga_d2m          0.028736
Malaga_t2m          0.019524
Malaga_i10fg        0.002526
Malaga_sp           0.019974
Malaga_tc

ValueError: feature_names mismatch: ['Madrid_d2m', 'Madrid_t2m', 'Madrid_i10fg', 'Madrid_sp', 'Madrid_tcc', 'Madrid_tp', 'Barcelona_d2m', 'Barcelona_t2m', 'Barcelona_i10fg', 'Barcelona_sp', 'Barcelona_tcc', 'Barcelona_tp', 'Valencia_d2m', 'Valencia_t2m', 'Valencia_i10fg', 'Valencia_sp', 'Valencia_tcc', 'Valencia_tp', 'Seville_d2m', 'Seville_t2m', 'Seville_i10fg', 'Seville_sp', 'Seville_tcc', 'Seville_tp', 'Zaragoza_d2m', 'Zaragoza_t2m', 'Zaragoza_i10fg', 'Zaragoza_sp', 'Zaragoza_tcc', 'Zaragoza_tp', 'Malaga_d2m', 'Malaga_t2m', 'Malaga_i10fg', 'Malaga_sp', 'Malaga_tcc', 'Malaga_tp', 'hour', 'weekday', 'month', 'business hour'] ['6070']
expected Zaragoza_t2m, Barcelona_t2m, Barcelona_d2m, Madrid_tp, Valencia_t2m, Valencia_sp, Seville_d2m, Seville_t2m, Madrid_t2m, business hour, Barcelona_tp, Malaga_i10fg, Malaga_tp, Malaga_t2m, Valencia_tcc, Valencia_d2m, Madrid_i10fg, Zaragoza_d2m, month, Seville_tcc, Madrid_tcc, Seville_sp, hour, Barcelona_i10fg, Madrid_sp, Barcelona_sp, Zaragoza_i10fg

In [ ]:
mse, mape, acc = xgb_ph.evaluate(X_test, y_test)

print("\n===================")
print("MSE: ", mse)
print("MAPE: ", mape)
print("Acc.: ", acc)